In [1]:
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from msrest.authentication import ApiKeyCredentials
from matplotlib import pyplot as plt
from PIL import Image, ImageDraw, ImageFont
import numpy as np
import os
import io
import gradio as gr

c:\Users\chch4\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Authenticate client

prediction_endpoint = "" # 본인 것으로 채워넣기
prediction_key = "" # 본인 것으로 채워넣기
project_id = "" # 본인 것으로 채워넣기
model_name = "Iteration1"

In [3]:
credentials = ApiKeyCredentials(in_headers={"Prediction-key": prediction_key})
predictor = CustomVisionPredictionClient(endpoint=prediction_endpoint, credentials=credentials)

In [4]:
def predict_image(input_img):
    h, w, ch= np.array(input_img).shape
    img_byte_arr = io.BytesIO()
    input_img.save(img_byte_arr, format='png')
    img_bytes = img_byte_arr.getvalue()
    results = predictor.detect_image(project_id, model_name, img_bytes)

    fig = plt.figure(figsize=(8,8))
    plt.axis('off')

    draw = ImageDraw.Draw(input_img)
    lineWidth = int(w/100)
    color = 'magenta'

    for prediction in results. predictions:
        if (prediction.probability*100) > 50:
            left = prediction.bounding_box.left * w
            top = prediction.bounding_box.top * h
            width = prediction.bounding_box.width * w
            height = prediction.bounding_box.height * h

            points = ((left,top), (left+width,top), (left+width,top+height), (left,top+height),(left,top))
            draw.line(points, fill=color, width=lineWidth)
            plt.annotate(prediction.tag_name + ' {0:.2f}%'.format(prediction.probability * 100), (left, top), color=color)

    plt.imshow(input_img)
    
    return plt

In [ ]:
demo = gr.Interface(fn=predict_image, inputs=gr.Image(type='pil'), outputs='plot')
demo.launch(share=True)